#  Capstone Project - The Battle of Neighbourhoods


### Introduction
Where would you recommend an art adventurer to visit in order to fulfill her/his hunger for various art pieces? The world is full of wonders and maybe the most ambiguous ones are considered as art. Many people dedicated themselves to travel around the world and discover these ambiguities. 
The data used in this project is provided by Foursquare location data. The data are grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, and stores which in this area. 


This project used the map visualizing technique and clustering of data return different number of clusters with similar results. The best result is as presented. However, I believe that the results are quite satisfying.

### Importing Library

In [39]:
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
import json
from IPython.display import Image 
from IPython.core.display import HTML 

# FourSquare Api Credentaials

In [40]:
ClIENT_ID = 'FWZNQR0AVXDKEK4FTMVASEBCPKFY11RMV03NZTGH5OLVH4NZ' # your Foursquare ID
ClIENT_SECRET = 'D0JCFVKFVXDGHIDQDAQMFOYI1FY4FYWRVC0WSHYBDTHGIMC0' # your Foursquare Secret
VERSION = '20200605'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: FWZNQR0AVXDKEK4FTMVASEBCPKFY11RMV03NZTGH5OLVH4NZ
Foursquare_Secret:D0JCFVKFVXDGHIDQDAQMFOYI1FY4FYWRVC0WSHYBDTHGIMC0


## DATA: 
In order to accomplish this goal, latitudes and longitudes of the hotels, restaurants, cafes, and stores are required. The "simplemaps.com" offers a simple, accurate and up-to-date database of the world's cities and their locations. From this data, I need to select the 'Toronto' cities and using their latitude and longitude values, I need to explore the venues with "art" section around these places by utilizing Foursquare API. 

The "simple map" data contains name, corresponding latitude, longitude, postal code, address, state and category along with many other features for all place in the Toronto. I only use the features that I mentioned in this data and then filtered the for each places. 

Then I used the acquired location data to explore the nearby art venues from the Foursquare API. 
Using the "explore" option, I look for top 25 art venues in 10 km radius for each city center and I get 
"Venue", "Venue Latitude", "Venue Longitude" and "Venue Category" columns with 3316 row in total. 


In [41]:
city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


# Creating Data Frame for Hotel

In [42]:
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FWZNQR0AVXDKEK4FTMVASEBCPKFY11RMV03NZTGH5OLVH4NZ&client_secret=D0JCFVKFVXDGHIDQDAQMFOYI1FY4FYWRVC0WSHYBDTHGIMC0&ll=43.6534817,-79.3839347&v=20200605&query=Hotel&radius=500&limit=30'

In [43]:
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-43-bc0f259b5133>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1606730495,False,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,NaN
1,51d212c3498ebf27dc469bc9,Chelsea Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1606730495,False,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,NaN
2,4af96fbbf964a520c01122e3,One King West Hotel & Residence,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1606730495,False,1 King St W,at Yonge St.,43.649139,-79.377876,"[{'label': 'display', 'lat': 43.6491395, 'lng'...",686,M5H 1A1,CA,Toronto,ON,Canada,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...",NaN,NaN
3,4ae61cf6f964a520caa421e3,Pantages Hotel & Spa,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1606730495,False,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,NaN
4,4b68aed1f964a520de862be3,The Rex Hotel Jazz & Blues Bar,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",v-1606730495,False,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",499,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",62225795,NaN


In [44]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]
clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",324,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,4ab2d511f964a5209b6c20e3
1,Chelsea Hotel,Hotel,33 Gerrard Street West,at Yonge St,43.658498,-79.383097,"[{'label': 'display', 'lat': 43.65849759157591...",562,M5G 1Z4,CA,Toronto,ON,Canada,"[33 Gerrard Street West (at Yonge St), Toronto...",NaN,51d212c3498ebf27dc469bc9
2,One King West Hotel & Residence,Hotel,1 King St W,at Yonge St.,43.649139,-79.377876,"[{'label': 'display', 'lat': 43.6491395, 'lng'...",686,M5H 1A1,CA,Toronto,ON,Canada,"[1 King St W (at Yonge St.), Toronto ON M5H 1A...",NaN,4af96fbbf964a520c01122e3
3,Pantages Hotel & Spa,Hotel,200 Victoria St,at Shuter St,43.654498,-79.379035,"[{'label': 'display', 'lat': 43.65449797039222...",410,NaN,CA,Toronto,ON,Canada,"[200 Victoria St (at Shuter St), Toronto ON, C...",NaN,4ae61cf6f964a520caa421e3
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",499,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",NaN,4b68aed1f964a520de862be3


In [45]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
3,Pantages Hotel & Spa,Hotel,200 Victoria St,43.654498,-79.379035,NaN,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON


In [46]:
# deleting rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3.head()

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
4,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON


In [47]:
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
8,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
10,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
24,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
27,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON


In [48]:
# deleting rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,Chelsea Hotel,Hotel,33 Gerrard Street West,43.658498,-79.383097,M5G 1Z4,ON
2,One King West Hotel & Residence,Hotel,1 King St W,43.649139,-79.377876,M5H 1A1,ON
6,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
7,Cosmopolitan Toronto Centre Hotel & Spa,Hotel,8 Colborne St,43.649064,-79.377598,M5E 1E1,ON
8,DoubleTree by Hilton Hotel Toronto Downtown,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
10,St Regis Hotel,Hotel,325 Bay St,43.649770,-79.380701,M5H 4G3,ON
24,The Adelaide Hotel Toronto,Hotel,325 Bay St,43.649831,-79.380164,M5H 4G3,ON
27,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON


In [49]:
# choosing the hotel which has the same postalCode with the event space
df_hotel = df_hotels[df_hotels.postalCode == 'M5H 2M9']
df_hotel

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.38453,M5H 2M9,ON


# Creating Data Frame for Park

In [50]:
# searching for Parks
search_query = 'Park'
radius = 10000

# Defining the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FWZNQR0AVXDKEK4FTMVASEBCPKFY11RMV03NZTGH5OLVH4NZ&client_secret=D0JCFVKFVXDGHIDQDAQMFOYI1FY4FYWRVC0WSHYBDTHGIMC0&ll=43.6534817,-79.3839347&v=20200605&query=Park&radius=10000&limit=30'

In [51]:
presults = requests.get(url).json()
venues = presults['response']['venues']
park_dataframe = json_normalize(venues)
park_dataframe.head()

<ipython-input-51-6d026bc5e16c>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  park_dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode
0,4bfc5019da7120a132df49fd,St. Andrews Playground / Dog Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1606730497,False,450 Adelaide Street West,Brant St & Adelaide St W,43.647388,-79.398752,"[{'label': 'display', 'lat': 43.64738759338116...",1372,CA,Toronto,ON,Canada,[450 Adelaide Street West (Brant St & Adelaide...,NaN
1,4b170e47f964a5208cc123e3,Queen's Park Subway Station,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",v-1606730497,False,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",M5G 2P1
2,4b54deadf964a520a3d027e3,Grange Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1606730497,False,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN
3,4bb79860cf2fc9b616779e02,Berczy Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1606730497,False,35 Wellington St. East,NaN,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",929,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",NaN
4,4b9d206bf964a520e69136e3,Queen's Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",v-1606730497,False,University Ave.,at Wellesley Ave.,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",1340,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...",M5R 2E8


In [52]:
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]
clean_park_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,id
0,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,Brant St & Adelaide St W,43.647388,-79.398752,"[{'label': 'display', 'lat': 43.64738759338116...",1372,CA,Toronto,ON,Canada,[450 Adelaide Street West (Brant St & Adelaide...,NaN,4bfc5019da7120a132df49fd
1,Queen's Park Subway Station,Metro Station,691 University Ave,at College St,43.660006,-79.390272,"[{'label': 'display', 'lat': 43.66000635700919...",887,CA,Toronto,ON,Canada,"[691 University Ave (at College St), Toronto O...",M5G 2P1,4b170e47f964a5208cc123e3
2,Grange Park,Park,NaN,NaN,43.652488,-79.392053,"[{'label': 'display', 'lat': 43.65248755299856...",663,CA,Toronto,ON,Canada,"[btwn College, University, Queen & Spadina, To...",NaN,4b54deadf964a520a3d027e3
3,Berczy Park,Park,35 Wellington St. East,NaN,43.648048,-79.375172,"[{'label': 'display', 'lat': 43.64804835388851...",929,CA,Toronto,ON,Canada,"[35 Wellington St. East, Toronto ON, Canada]",NaN,4bb79860cf2fc9b616779e02
4,Queen's Park,Park,University Ave.,at Wellesley Ave.,43.663946,-79.392180,"[{'label': 'display', 'lat': 43.66394609897775...",1340,CA,Toronto,ON,Canada,"[University Ave. (at Wellesley Ave.), Toronto ...",M5R 2E8,4b9d206bf964a520e69136e3


In [53]:
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2.head()

,name,categories,address,lat,lng,state,postalCode
0,St. Andrews Playground / Dog Park,Park,450 Adelaide Street West,43.647388,-79.398752,ON,NaN
1,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,ON,M5G 2P1
2,Grange Park,Park,NaN,43.652488,-79.392053,ON,NaN
3,Berczy Park,Park,35 Wellington St. East,43.648048,-79.375172,ON,NaN
4,Queen's Park,Park,University Ave.,43.663946,-79.392180,ON,M5R 2E8


In [54]:
# deleting rows with none values
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3.head()

,name,categories,address,lat,lng,state,postalCode
1,Queen's Park Subway Station,Metro Station,691 University Ave,43.660006,-79.390272,ON,M5G 2P1
4,Queen's Park,Park,University Ave.,43.663946,-79.392180,ON,M5R 2E8
6,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,ON,M5T 2N4
7,Physiomed College Park,Medical Center,382 Yonge St,43.659240,-79.382860,ON,M5B 1S8
8,One Park Lane,Residential Building (Apartment / Condo),280 Simcoe St,43.655442,-79.390056,ON,M5T 2Y5


In [55]:
# deleting rows which its category is not Park
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park.head()

,name,categories,address,lat,lng,state,postalCode
4,Queen's Park,Park,University Ave.,43.663946,-79.392180,ON,M5R 2E8
6,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,ON,M5T 2N4
10,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,ON,M5H 2N2
12,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,ON,M5V 3Z1
15,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,ON,M5A 2H3


# Creating Data Frame for Restaurant

In [56]:
# searching for Restaurants
search_query = 'Restaurant'
radius = 10000

# Defining the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FWZNQR0AVXDKEK4FTMVASEBCPKFY11RMV03NZTGH5OLVH4NZ&client_secret=D0JCFVKFVXDGHIDQDAQMFOYI1FY4FYWRVC0WSHYBDTHGIMC0&ll=43.6534817,-79.3839347&v=20200605&query=Restaurant&radius=10000&limit=30'

In [57]:
Rresults = requests.get(url).json()
venues = Rresults['response']['venues']
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

<ipython-input-57-9accf9f220c3>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Restaurant_dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1606730498,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,5c33c7cd8194fc002c66f72e,Green Tea Restaurant Downtown,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1606730498,False,261 Spadina Avenue. Upper level,43.652488,-79.397501,"[{'label': 'display', 'lat': 43.65248783033554...",1098,M5T 2E3,CA,Toronto,ON,Canada,"[261 Spadina Avenue. Upper level, Toronto ON M...",NaN,NaN,NaN
2,5750b013498e755287c6de97,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",v-1606730498,False,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,NaN
3,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1606730498,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
4,4ada5d5bf964a520e92121e3,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1606730498,False,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,NaN


In [58]:
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]
clean_Restaurant_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,4ad4c05ff964a52048f720e3
1,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,"[{'label': 'display', 'lat': 43.65248783033554...",1098,M5T 2E3,CA,Toronto,ON,Canada,"[261 Spadina Avenue. Upper level, Toronto ON M...",NaN,NaN,5c33c7cd8194fc002c66f72e
2,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,5750b013498e755287c6de97
3,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,NaN,4ad4c05cf964a52006f620e3
4,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,4ada5d5bf964a520e92121e3


In [59]:
#deleting unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2.head()

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,M5T 2E3,ON
2,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,NaN,ON
3,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
4,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON


In [60]:
# deleting rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant.head()

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Green Tea Restaurant Downtown,Chinese Restaurant,261 Spadina Avenue. Upper level,43.652488,-79.397501,M5T 2E3,ON
3,Victoria's Restaurant,Restaurant,37 King Street East,43.649298,-79.376431,M5C 1E9,ON
4,The Hot House Restaurant & Bar,American Restaurant,35 Church St,43.648824,-79.373702,M5E 1T3,ON
5,Studio Restaurant,Breakfast Spot,389 Church St.,43.661500,-79.379319,M5B,ON


# Creating Data Frame for Cafeteria

In [61]:
# searching for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Defining the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FWZNQR0AVXDKEK4FTMVASEBCPKFY11RMV03NZTGH5OLVH4NZ&client_secret=D0JCFVKFVXDGHIDQDAQMFOYI1FY4FYWRVC0WSHYBDTHGIMC0&ll=43.6534817,-79.3839347&v=20200605&query=Cafeteria&radius=10000&limit=30'

In [62]:
cresults = requests.get(url).json()
venues = cresults['response']['venues']
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

<ipython-input-62-0fbedda48d2f>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Cafeteria_dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4f9165bde4b04fef0087e4e2,Cafeteria,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1606730499,False,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",448,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN
1,4b57256bf964a520332828e3,Sears Cafeteria,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1606730499,False,Yonge,43.656038,-79.380672,"[{'label': 'display', 'lat': 43.65603785909391...",387,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON, Canada]",Dundas,NaN
2,53bd6dfe498e6c1d3bc9333e,Industrial Alliance Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1606730499,False,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",505,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
3,524f1c8f498e983dde61ada0,Chestnut Tree Cafeteria,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",v-1606730499,False,89 Chestnut Street,43.648231,-79.384045,"[{'label': 'display', 'lat': 43.648231, 'lng':...",584,CA,Toronto,ON,Canada,"[89 Chestnut Street, Toronto ON, Canada]",NaN,NaN
4,556f826b498e07f469e6bf1f,Dears Cafeteria,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1606730499,False,NaN,43.654179,-79.392555,"[{'label': 'display', 'lat': 43.65417861938476...",698,CA,NaN,NaN,Canada,[Canada],NaN,NaN


In [63]:
# keeping only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]
clean_Cafeteria_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,"[{'label': 'display', 'lat': 43.65720930549997...",448,CA,Toronto,ON,Canada,"[The Hospital for Sick Children (SickKids), To...",NaN,NaN,4f9165bde4b04fef0087e4e2
1,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,"[{'label': 'display', 'lat': 43.65603785909391...",387,CA,Toronto,ON,Canada,"[Yonge (Dundas), Toronto ON, Canada]",Dundas,NaN,4b57256bf964a520332828e3
2,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,"[{'label': 'display', 'lat': 43.655657, 'lng':...",505,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,53bd6dfe498e6c1d3bc9333e
3,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,"[{'label': 'display', 'lat': 43.648231, 'lng':...",584,CA,Toronto,ON,Canada,"[89 Chestnut Street, Toronto ON, Canada]",NaN,NaN,524f1c8f498e983dde61ada0
4,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,"[{'label': 'display', 'lat': 43.65417861938476...",698,CA,NaN,NaN,Canada,[Canada],NaN,NaN,556f826b498e07f469e6bf1f


In [64]:
# deleting unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2.head()

,name,categories,address,lat,lng,state,postalCode
0,Cafeteria,Fast Food Restaurant,The Hospital for Sick Children (SickKids),43.657209,-79.386063,ON,NaN
1,Sears Cafeteria,Food Court,Yonge,43.656038,-79.380672,ON,NaN
2,Industrial Alliance Cafeteria,Cafeteria,NaN,43.655657,-79.389443,ON,NaN
3,Chestnut Tree Cafeteria,Cafeteria,89 Chestnut Street,43.648231,-79.384045,ON,NaN
4,Dears Cafeteria,Dessert Shop,NaN,43.654179,-79.392555,NaN,NaN


In [65]:
# deleting rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,state,postalCode
8,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,ON,M4V 1H7
18,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,ON,M5B 1W8
20,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,ON,M5B 2K3
25,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,ON,m3c 3k7
26,Innis College,Student Center,2 Sussex Ave,43.665556,-79.399298,ON,M5S 1J5


# Creating Data Frame for Shopping

In [66]:
search_query = 'Shopping'
radius = 1000

# Defining the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FWZNQR0AVXDKEK4FTMVASEBCPKFY11RMV03NZTGH5OLVH4NZ&client_secret=D0JCFVKFVXDGHIDQDAQMFOYI1FY4FYWRVC0WSHYBDTHGIMC0&ll=43.6534817,-79.3839347&v=20200605&query=Shopping&radius=1000&limit=30'

In [67]:
sresults = requests.get(url).json()
venues = sresults['response']['venues']
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

<ipython-input-67-fc82f912e8a4>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Shopping_dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,5840cafe06f1a34af80cc609,Saks Fifth Avenue Club - Personal Shopping,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1606730500,False,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN
1,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1606730500,False,220 Yonge St,btwn Queen & Dundas,43.654447,-79.380952,NaN,263,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON"
2,4b9e86dff964a520e6eb36e3,TD Centre Shopping Concourse,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1606730500,False,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN


In [68]:
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]
clean_Shopping_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN,5840cafe06f1a34af80cc609
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,btwn Queen & Dundas,43.654447,-79.380952,NaN,263,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN,4b9e86dff964a520e6eb36e3


In [69]:
# deleting unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654447,-79.380952,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


In [70]:
# deleting rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
1,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654447,-79.380952,M5B 2H1,ON
2,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON


# Map to Visualize Hotels, Shopping Stores and Cafeteria and lets see how they cluster together

In [71]:
# creating dataframe of hotels, shopping stores and Cafeteria
hotel_neighbourhood_df = pd.concat([df_hotel, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df.head()

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,De La Salle College Cafeteria,College Cafeteria,131 Farnham Avenue,43.683003,-79.397815,M4V 1H7,ON
2,Marketeria,Restaurant,30 Bond St.,43.653585,-79.378843,M5B 1W8,ON
3,The Hub,College Cafeteria,350 Victoria St.,43.658585,-79.380622,M5B 2K3,ON
4,6th Floor Cafeteria 12 Concorde Place,Café,12 Concorde Place,43.721861,-79.329339,m3c 3k7,ON


In [72]:
# Generating map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

# Map to Visualize Park, Restaurant and Cafeteria and lets see how they cluster together

In [73]:
# creating dataframe of Park, Restaurant and Cafeteria
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df.head()

,name,categories,address,lat,lng,state,postalCode
0,Queen's Park,Park,University Ave.,43.663946,-79.392180,ON,M5R 2E8
1,Bellevue Square Park,Park,btwn Bellevue & Augusta Ave.,43.653610,-79.402199,ON,M5T 2N4
2,Trinity Bellwoods Park,Park,1053 Dundas St. W.,43.647072,-79.413756,ON,M5H 2N2
3,Canoe Landing Park,Park,50 Fort York Blvd,43.638762,-79.397067,ON,M5V 3Z1
4,Riverdale Park West,Park,500 Gerrard St.,43.666048,-79.360941,ON,M5A 2H3


In [74]:
# Generating map to visualize park neighbourhood including Restaurant and Cafeteria 
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map

###### CONCLUSION: 
To sum up, the adventurer can go to first map shows the different park restaurant and cafeteria and second is show the hotels and shopping stores.